In [1]:
import pandas as pd
import ta 
import optuna 
import time
import numpy as np
from multiprocessing import Pool
from itertools import combinations, chain 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [2]:
data_1m = pd.read_csv("aapl_1d_train.csv")
data_1m = data_1m.dropna()

In [3]:
def powerset(s):
    return chain.from_iterable(combinations(s,r) for r in range(1,len(s)+1))

In [4]:
def file_features(data, ds_type: str):
    data1=pd.DataFrame()
    #Calcular indicadores tecnicos
    cmf_data = ta.volume.ChaikinMoneyFlowIndicator(data.High, data.Low, data.Close, data.Volume, window = 14)
    rsi_data = ta.momentum.RSIIndicator(data.Close, window=14)
    
    data1["CMF"] = cmf_data.chaikin_money_flow()
    data1["RSI"] = rsi_data.rsi()
    # Calcular la volatilidad
    data1['Volatility'] = data['High'] - data['Low']
    data1['Close_Lag0'] = data['Close']
    # Calcular las tendencias
    for i in range(1, 5 + 1):
        data1[f'Close_Lag{i}'] = data['Close'].shift(i)
    #Variable ded respuesta
    if ds_type == "buy":
        data1['Response'] = (data['Close'] < data['Close'].shift(-10))
    else:
        data1['Response'] = (data['Close'] > data['Close'].shift(-10))
    
    data1 = data1.drop(data1.index[:30])
    data1 = data1.drop(data1.index[-30:])
    data1.reset_index(drop=True, inplace=True)
    
    return data1

In [5]:
dataresult_long_1d = file_features(data_1m, ds_type="buy")
dataresult_short_1d = file_features(data_1m, ds_type="sell")
dataresult_short_1d

,CMF,RSI,Volatility,Close_Lag0,Close_Lag1,Close_Lag2,Close_Lag3,Close_Lag4,Close_Lag5,Response
0,0.246933,61.064016,4.7488,119.0000,115.8080,108.7520,113.5008,112.6272,114.8784,False
1,0.223372,55.142449,6.0032,114.1280,119.0000,115.8080,108.7520,113.5008,112.6272,False
2,0.314459,55.852561,2.3856,114.8784,114.1280,119.0000,115.8080,108.7520,113.5008,False
3,0.171494,51.598684,4.4912,111.2496,114.8784,114.1280,119.0000,115.8080,108.7520,False
4,0.254775,54.250846,10.2480,113.8144,111.2496,114.8784,114.1280,119.0000,115.8080,False
...,...,...,...,...,...,...,...,...,...,...
5219,0.040511,45.850110,3.1900,119.4900,115.9700,116.3200,118.6900,119.0300,114.9500,True
5220,0.035327,45.534960,1.9600,119.2100,119.4900,115.9700,116.3200,118.6900,119.0300,True
5221,0.049474,45.606858,1.8000,119.2600,119.2100,119.4900,115.9700,116.3200,118.6900,True
5222,0.049731,47.169059,2.8400,120.3000,119.2600,119.2100,119.4900,115.9700,116.3200,False


In [6]:
def objective_log_regresor(trial, data):
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Definir los parámetros a optimizar
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    C = trial.suggest_loguniform('C', 0.001, 1000)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])

    # Crear el modelo de regresión logística con los parámetros sugeridos
    model = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=10_000, random_state=123)
    # Entrenar el modelo
    model.fit(X_train, y_train)
    # Calcular la precisión en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [7]:
def objective_svm(trial, data):
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Definir los parámetros a optimizar
    C = trial.suggest_loguniform('C', 0.001, 1000)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)
    else:
        degree = 3  # Valor predeterminado si el kernel no es 'poly'
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto']) if kernel in ['rbf', 'poly', 'sigmoid'] else 'scale'
    # Crear el modelo SVM con los parámetros sugeridos
    model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, max_iter=100_000, random_state=123)
    # Entrenar el modelo
    model.fit(X_train, y_train)
    # Calcular la precisión en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [8]:
def objective_xgboost(trial, data):
    data = data.copy()
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    # Definir los parámetros a optimizar
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, step=100)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
    subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
    colsample_bytree = trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1.0, 0.1)
    # Crear el modelo XGBoost con los parámetros sugeridos
    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=123
    )
    # Entrenar el modelo
    model.fit(X_train, y_train)
    # Calcular la precisión en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [9]:
def optimize_params_log_regresor(data):
    # Crear un estudio Optuna para la optimización
    study = optuna.create_study(direction='maximize')
    
    # Función objetivo con el dataset como parámetro fijo
    objective_fn = lambda trial: objective_log_regresor(trial, data)
    
    # Ejecutar la optimización
    study.optimize(objective_fn, n_trials=2)

    # Obtener los mejores parámetros
    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

In [10]:
def optimize_params_svm(data):
    # Crear un estudio Optuna para la optimización
    study = optuna.create_study(direction='maximize')
    
    # Función objetivo con el dataset como parámetro fijo
    objective_fn = lambda trial: objective_svm(trial, data)
    
    # Ejecutar la optimización
    study.optimize(objective_fn, n_trials=2)

    # Obtener los mejores parámetros
    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

In [11]:
def optimize_params_xgboost(data):
    # Crear un estudio Optuna para la optimización
    study = optuna.create_study(direction='maximize')
    
    # Función objetivo con el dataset como parámetro fijo
    objective_fn = lambda trial: objective_xgboost(trial, data)
    
    # Ejecutar la optimización
    study.optimize(objective_fn, n_trials=2)

    # Obtener los mejores parámetros
    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

In [12]:
def optimize_params(data):
    # Optimización de regresión logística
    best_params_lr, best_accuracy_lr = optimize_params_log_regresor(data)
    print("Mejores parámetros de regresión logística:", best_params_lr)
    print("Precisión del modelo de regresión logística:", best_accuracy_lr)
    # Optimización de SVM
    best_params_svm, best_accuracy_svm = optimize_params_svm(data)
    print("Mejores parámetros de SVM:", best_params_svm)
    print("Precisión del modelo de SVM:", best_accuracy_svm)
    # Optimización de XGBoost
    best_params_xgb, best_accuracy_xgb = optimize_params_xgboost(data)
    print("Mejores parámetros de XGBoost:", best_params_xgb)
    print("Precisión del modelo de XGBoost:", best_accuracy_xgb)

In [13]:
params_1d_long = optimize_params(dataresult_long_1d)
params_1d_short = optimize_params(dataresult_short_1d)

[I 2024-03-08 09:12:46,743] A new study created in memory with name: no-name-9b46ecb1-adc0-4dc4-b61a-daa4c24e8b23
C:\Users\elias\AppData\Local\Temp\ipykernel_5256\2368889626.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-03-08 09:12:53,927] Trial 0 finished with value: 0.6564593301435406 and parameters: {'penalty': 'l2', 'C': 3.0941148148710647, 'solver': 'saga'}. Best is trial 0 with value: 0.6564593301435406.
C:\Users\elias\AppData\Local\Temp\ipykernel_5256\2368889626.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
[I 2024-03-08 09:12:55,366] Trial 1 finish

Mejores parámetros de regresión logística: {'penalty': 'l1', 'C': 406.4481314429395, 'solver': 'liblinear'}
Precisión del modelo de regresión logística: 0.6583732057416268


[I 2024-03-08 09:12:56,475] Trial 0 finished with value: 0.6593301435406699 and parameters: {'C': 211.57942168954827, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.6593301435406699.
C:\Users\elias\AppData\Local\Temp\ipykernel_5256\744662071.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
C:\Users\elias\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-08 09:13:00,973] Trial 1 finished with value: 0.5827751196172248 and parameters: {'C': 0.38855981209948753, 'kernel': 'linear'}. Best is trial 0 with value: 0.6593301435406699.
[I 2024-03-08 09:13:00,973] A new study created in memory with name

Mejores parámetros de SVM: {'C': 211.57942168954827, 'kernel': 'sigmoid', 'gamma': 'auto'}
Precisión del modelo de SVM: 0.6593301435406699


[I 2024-03-08 09:13:04,005] Trial 0 finished with value: 0.5722488038277512 and parameters: {'n_estimators': 600, 'max_depth': 8, 'learning_rate': 0.015902770191447604, 'subsample': 1.0, 'colsample_bytree': 0.5}. Best is trial 0 with value: 0.5722488038277512.
C:\Users\elias\AppData\Local\Temp\ipykernel_5256\275342267.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
C:\Users\elias\AppData\Local\Temp\ipykernel_5256\275342267.py:12: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
C:\Users\elias\AppData\Local\Temp\ipykernel_525

...

Mejores parámetros de regresión logística: {'penalty': 'l2', 'C': 115.21108890139607, 'solver': 'liblinear'}
Precisión del modelo de regresión logística: 0.661244019138756


[I 2024-03-08 09:13:10,830] Trial 0 finished with value: 0.661244019138756 and parameters: {'C': 0.8253700439720046, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.661244019138756.
C:\Users\elias\AppData\Local\Temp\ipykernel_5256\744662071.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 1000)
C:\Users\elias\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-08 09:13:12,759] Trial 1 finished with value: 0.6086124401913876 and parameters: {'C': 0.0160161091000553, 'kernel': 'linear'}. Best is trial 0 with value: 0.661244019138756.
[I 2024-03-08 09:13:12,759] A new study created in memory with name: no

Mejores parámetros de SVM: {'C': 0.8253700439720046, 'kernel': 'sigmoid', 'gamma': 'auto'}
Precisión del modelo de SVM: 0.661244019138756


[I 2024-03-08 09:13:14,078] Trial 0 finished with value: 0.570334928229665 and parameters: {'n_estimators': 600, 'max_depth': 7, 'learning_rate': 0.0441689761809396, 'subsample': 0.7, 'colsample_bytree': 0.7}. Best is trial 0 with value: 0.570334928229665.
C:\Users\elias\AppData\Local\Temp\ipykernel_5256\275342267.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
C:\Users\elias\AppData\Local\Temp\ipykernel_5256\275342267.py:12: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  subsample = trial.suggest_discrete_uniform('subsample', 0.5, 1.0, 0.1)
C:\Users\elias\AppData\Local\Temp\ipykernel_5256\27

Mejores parámetros de XGBoost: {'n_estimators': 600, 'max_depth': 7, 'learning_rate': 0.0441689761809396, 'subsample': 0.7, 'colsample_bytree': 0.7}
Precisión del modelo de XGBoost: 0.570334928229665


In [14]:
def buy_signals(data):
    buy_signals = pd.DataFrame()
    # Selecciona las características
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]

    # Crear modelos con los mejores parámetros encontrados para cada algoritmo
    best_logistic_model = LogisticRegression(penalty='l1', C=142.00912335775166, solver='liblinear')
    best_svm_model = SVC(C=1.0, kernel='rbf', gamma='scale')
    best_xgboost_model = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, subsample=0.8, colsample_bytree=0.8)

    # Entrenar los modelos con todo el conjunto de datos original
    best_logistic_model.fit(X, y)
    best_svm_model.fit(X, y)
    best_xgboost_model.fit(X, y)

    # Realizar predicciones en el conjunto de datos original
    predictions_lr = best_logistic_model.predict(X)
    predictions_svm = best_svm_model.predict(X)
    predictions_xgboost = best_xgboost_model.predict(X)

    # Agregar las predicciones como nuevas columnas al conjunto de datos original
    buy_signals['predicciones_lr'] = predictions_lr
    buy_signals['predicciones_svm'] = predictions_svm
    buy_signals['predicciones_xgboost'] = predictions_xgboost
    
    return buy_signals

In [15]:
def sell_signals(data):
    sell_signals = pd.DataFrame()
    # Selecciona las características
    X = data.iloc[:, :-1]
    # Selecciona la variable objetivo
    y = data.iloc[:, -1]

    # Crear modelos con los mejores parámetros encontrados para cada algoritmo
    best_logistic_model = LogisticRegression(penalty='l1', C=142.00912335775166, solver='liblinear')
    best_svm_model = SVC(C=1.0, kernel='rbf', gamma='scale')
    best_xgboost_model = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, subsample=0.8, colsample_bytree=0.8)

    # Entrenar los modelos con todo el conjunto de datos original
    best_logistic_model.fit(X, y)
    best_svm_model.fit(X, y)
    best_xgboost_model.fit(X, y)

    # Realizar predicciones en el conjunto de datos original
    predictions_lr = best_logistic_model.predict(X)
    predictions_svm = best_svm_model.predict(X)
    predictions_xgboost = best_xgboost_model.predict(X)
    predictions_xgboost_bool = predictions_xgboost.astype(bool)

    # Agregar las predicciones como nuevas columnas al conjunto de datos original
    sell_signals['predicciones_lr'] = predictions_lr
    sell_signals['predicciones_svm'] = predictions_svm
    sell_signals['predicciones_xgboost'] = predictions_xgboost_bool
    
    return sell_signals

In [16]:
global_buy_signals = buy_signals(dataresult_long_1d)
global_sell_signals = sell_signals(dataresult_short_1d)

In [17]:
def backtest(data, buy_signals, sell_signals, stop_loss, take_profit, n_shares):
    history = []
    active_operations = []
    cash = 1_000_000
    com = 1.25 / 100
    
    for i, row in data.iterrows():
        # close active operation
        active_op_temp = []
        for operation in active_operations:
            if operation["stop_loss"] > row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
            elif operation["take_profit"] < row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
            else:
                active_op_temp.append(operation)
        active_operations = active_op_temp

        # check if we have enough cash
        if cash < (row.Close * (1 + com)):
            asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
            portfolio_value = cash + asset_vals
            continue

        # Apply buy signals
        if buy_signals.loc[i].any():
            active_operations.append({
                "bought": row.Close,
                "n_shares": n_shares,
                "stop_loss": row.Close * stop_loss,
                "take_profit": row.Close * take_profit
            })

            cash -= row.Close * (1 + com) * n_shares

        # Apply sell signals
        if sell_signals.loc[i].any():
            active_op_temp = []
            for operation in active_operations:
                if operation["take_profit"] < row.Close or operation["stop_loss"] > row.Close:
                    cash += (row.Close * operation["n_shares"]) * (1 - com)
                else:
                    active_op_temp.append(operation)
            active_operations = active_op_temp

        asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
        portfolio_value = cash + asset_vals

    return portfolio_value

In [18]:
def optimize(trial, strategy, data):
    portfolio_value = 0

    stop_loss = trial.suggest_float("stop_loss", 0.00250, 0.05)
    take_profit = trial.suggest_float("take_profit", 0.00250, 0.05)
    n_shares = trial.suggest_int("n_shares", 5, 200)

    strat_params = {}

    buy_signals = pd.DataFrame()
    sell_signals = pd.DataFrame()

    if "logistic" in strategy:
        buy_signals["logistic"] = global_buy_signals["predicciones_lr"]
        sell_signals["logistic"] = global_sell_signals["predicciones_lr"]
        
    if "svm" in strategy:
        buy_signals["svm"] = global_buy_signals["predicciones_svm"]
        sell_signals["svm"] = global_sell_signals["predicciones_svm"]
        
    if "xg" in strategy:
        buy_signals["xg"] = global_buy_signals["predicciones_xgboost"]
        sell_signals["xg"] = global_sell_signals["predicciones_xgboost"]
    
    return backtest(data, buy_signals, sell_signals, stop_loss, take_profit, n_shares)

In [19]:
def optimize_file(data):
    data = data.drop(data.index[:30])
    data = data.drop(data.index[-30:])
    data.reset_index(drop=True, inplace=True)
    strategies = list(powerset(["logistic", "svm", "xg"]))
    best_strat = None
    best_val = -1
    best_params = None

    for strat in strategies:
        study = optuna.create_study(direction="maximize")
        study.optimize(lambda x: optimize(x, strat, data), n_trials=30)
        value = study.best_value
        if value > best_val:
            best_val = value
            best_strat = strat
            best_params = study.best_params
    print(study.best_value)
    print(best_strat)
    print(best_params)

    return {"file": data,
            "strat": best_strat,
            "value": best_val,
            "params": best_params}

In [20]:
file_1d = optimize_file(data_1m)

[I 2024-03-08 09:13:26,208] A new study created in memory with name: no-name-441721f4-b7e2-4a35-ab72-68b349b90df5
[I 2024-03-08 09:13:27,263] Trial 0 finished with value: -237.27334000318479 and parameters: {'stop_loss': 0.04999708328489621, 'take_profit': 0.03377754186393962, 'n_shares': 148}. Best is trial 0 with value: -237.27334000318479.
[I 2024-03-08 09:13:29,048] Trial 1 finished with value: 889487.5216749996 and parameters: {'stop_loss': 0.03102219801077408, 'take_profit': 0.014029475792409518, 'n_shares': 5}. Best is trial 1 with value: 889487.5216749996.
[I 2024-03-08 09:13:30,610] Trial 2 finished with value: 491642.5997050028 and parameters: {'stop_loss': 0.04842706532787188, 'take_profit': 0.0038350096953392895, 'n_shares': 23}. Best is trial 1 with value: 889487.5216749996.
[I 2024-03-08 09:13:31,800] Trial 3 finished with value: -915.4147999965426 and parameters: {'stop_loss': 0.013479176932453565, 'take_profit': 0.02175144681568588, 'n_shares': 140}. Best is trial 1 wit

[I 2024-03-08 09:14:12,021] Trial 5 finished with value: 29.927800007012593 and parameters: {'stop_loss': 0.036677693038635724, 'take_profit': 0.02918460073638147, 'n_shares': 72}. Best is trial 2 with value: 230067.701920009.
[I 2024-03-08 09:14:13,382] Trial 6 finished with value: 486711.8012800041 and parameters: {'stop_loss': 0.0344649265139408, 'take_profit': 0.032578879502215975, 'n_shares': 24}. Best is trial 6 with value: 486711.8012800041.
[I 2024-03-08 09:14:14,316] Trial 7 finished with value: -945.7441549996292 and parameters: {'stop_loss': 0.04918998099627867, 'take_profit': 0.014373119735151256, 'n_shares': 137}. Best is trial 6 with value: 486711.8012800041.
[I 2024-03-08 09:14:15,647] Trial 8 finished with value: 529485.8178399992 and parameters: {'stop_loss': 0.010852852992709288, 'take_profit': 0.015772685190204583, 'n_shares': 22}. Best is trial 8 with value: 529485.8178399992.
[I 2024-03-08 09:14:17,227] Trial 9 finished with value: 572259.8344000021 and parameters:

[I 2024-03-08 09:14:54,555] Trial 11 finished with value: 847187.3559699954 and parameters: {'stop_loss': 0.0033323255278491486, 'take_profit': 0.021869037503099483, 'n_shares': 9}. Best is trial 10 with value: 898124.9039799995.
[I 2024-03-08 09:14:55,779] Trial 12 finished with value: 915104.0866500016 and parameters: {'stop_loss': 0.002977556939392477, 'take_profit': 0.017531685640611652, 'n_shares': 5}. Best is trial 12 with value: 915104.0866500016.
[I 2024-03-08 09:14:57,003] Trial 13 finished with value: 32186.587810004687 and parameters: {'stop_loss': 0.0027935590247103776, 'take_profit': 0.015570295188278408, 'n_shares': 57}. Best is trial 12 with value: 915104.0866500016.
[I 2024-03-08 09:14:58,287] Trial 14 finished with value: 439686.9718900031 and parameters: {'stop_loss': 0.012515531085145963, 'take_profit': 0.015062819631676102, 'n_shares': 33}. Best is trial 12 with value: 915104.0866500016.
[I 2024-03-08 09:14:59,457] Trial 15 finished with value: -78.76926499693946 an

[I 2024-03-08 09:15:39,083] Trial 17 finished with value: -312.2960799960392 and parameters: {'stop_loss': 0.019995785698669655, 'take_profit': 0.043239958920243814, 'n_shares': 46}. Best is trial 13 with value: 887706.1369750006.
[I 2024-03-08 09:15:40,087] Trial 18 finished with value: -426.9132199961168 and parameters: {'stop_loss': 0.03503210588772071, 'take_profit': 0.03553215422202746, 'n_shares': 84}. Best is trial 13 with value: 887706.1369750006.
[I 2024-03-08 09:15:41,553] Trial 19 finished with value: 820329.8191600004 and parameters: {'stop_loss': 0.007784342911656755, 'take_profit': 0.04342857983669041, 'n_shares': 8}. Best is trial 13 with value: 887706.1369750006.
[I 2024-03-08 09:15:43,102] Trial 20 finished with value: -3935.2559750032206 and parameters: {'stop_loss': 0.028403921906782956, 'take_profit': 0.04745425856622254, 'n_shares': 45}. Best is trial 13 with value: 887706.1369750006.
[I 2024-03-08 09:15:44,444] Trial 21 finished with value: 865247.3643700005 and p

[I 2024-03-08 09:16:25,924] Trial 23 finished with value: 328519.9789600016 and parameters: {'stop_loss': 0.016946569028975657, 'take_profit': 0.01480428881479326, 'n_shares': 31}. Best is trial 18 with value: 891696.7707999995.
[I 2024-03-08 09:16:27,350] Trial 24 finished with value: 60.18696000252203 and parameters: {'stop_loss': 0.022894926238559946, 'take_profit': 0.02192623095839946, 'n_shares': 56}. Best is trial 18 with value: 891696.7707999995.
[I 2024-03-08 09:16:28,853] Trial 25 finished with value: 891696.7707999995 and parameters: {'stop_loss': 0.031399085555202655, 'take_profit': 0.007757579939527658, 'n_shares': 5}. Best is trial 18 with value: 891696.7707999995.
[I 2024-03-08 09:16:30,378] Trial 26 finished with value: 393501.91648000083 and parameters: {'stop_loss': 0.031096718326682676, 'take_profit': 0.008895981414341683, 'n_shares': 28}. Best is trial 18 with value: 891696.7707999995.
[I 2024-03-08 09:16:31,493] Trial 27 finished with value: 76.5536950069727 and par

[I 2024-03-08 09:17:14,985] Trial 29 finished with value: 86.63464000341628 and parameters: {'stop_loss': 0.03494797690272215, 'take_profit': 0.037992762630191336, 'n_shares': 78}. Best is trial 4 with value: 895122.7913750001.
[I 2024-03-08 09:17:14,985] A new study created in memory with name: no-name-ca98525a-5d89-4f17-b887-0b0e59e3199a
[I 2024-03-08 09:17:16,648] Trial 0 finished with value: 648151.9540800023 and parameters: {'stop_loss': 0.004579927385681394, 'take_profit': 0.03912926287668609, 'n_shares': 16}. Best is trial 0 with value: 648151.9540800023.
[I 2024-03-08 09:17:17,551] Trial 1 finished with value: -92.5636799918575 and parameters: {'stop_loss': 0.04211986832514111, 'take_profit': 0.04349593848439424, 'n_shares': 196}. Best is trial 0 with value: 648151.9540800023.
[I 2024-03-08 09:17:18,577] Trial 2 finished with value: -326.5186249954277 and parameters: {'stop_loss': 0.011103952621539858, 'take_profit': 0.03604944792391165, 'n_shares': 155}. Best is trial 0 with v

890047.4856499997
('xg',)
{'stop_loss': 0.002977556939392477, 'take_profit': 0.017531685640611652, 'n_shares': 5}
